In [1]:
import os
import re
import httpx

import edgar as edgar
import pandas as pd
from bs4 import BeautifulSoup
from rich import print
from dotenv import load_dotenv, find_dotenv

%load_ext rich

load_dotenv(find_dotenv())

edgar.set_identity("lakshya@insy695.com")

[11:00:09] INFO     Identity of the Edgar REST client set to [lakshya@insy695.com]                      core.py:153

In [8]:
BASE_URL = "https://api.sec-api.io/"
client = httpx.Client(
    base_url=BASE_URL,
    timeout=60.0
)


In [9]:
def parse_html_filing(filing):
    html_ = filing.html()
    regex = re.compile(r"(>Item(\s|&#160;|&nbsp;)(7A|7)\.{0,1})|(ITEM\s(7A|7)\.{0,1})")
    matches = regex.finditer(html_)

    match_df = pd.DataFrame([(x.group(), x.start(), x.end()) for x in matches])

    match_df.columns = ["item", "start", "end"]
    match_df["item"] = match_df.item.str.lower()

    match_df.replace("&#160;", " ", regex=True, inplace=True)
    match_df.replace("&nbsp;", " ", regex=True, inplace=True)
    match_df.replace(" ", " ", regex=True, inplace=True)
    match_df.replace("\.", "", regex=True, inplace=True)
    match_df.replace(">", "", regex=True, inplace=True)

    match_df = match_df.sort_values("start", ascending=True).drop_duplicates(
        subset=["item"], keep="last"
    )

    if len(match_df["start"].values) > 1:
        item_7_raw = html_[match_df["start"].values[0] : match_df["start"].values[1]]
    else:
        item_7_raw = html_[match_df["start"].values[0] :]

    item_7_content = BeautifulSoup(item_7_raw, "lxml").get_text("\n\n")

    return item_7_content

In [10]:
def extract_section(ticker, year):
    company = edgar.Company(ticker)
    filings_df = company.get_filings(form="10-K").to_pandas()
    filings_df["year"] = pd.to_datetime(filings_df["reportDate"]).dt.year
    accession_number = filings_df.query(f"year == {year}")["accession_number"].values[0]

    filing = edgar.get_by_accession_number(accession_number)

    item_7_content_html = parse_html_filing(filing)

    extractor_params = {
        "url": filing.url,
        "item": "7",
        "type": "text",
        "token": os.getenv("SEC_API_KEY"),
    }

    response = client.get("extractor", params=extractor_params)

    item_7_extracted = response.text

    # Save to files
    with open(f"extracted/{ticker}_{year}_item_7_extracted.txt", "w") as f:
        f.write(item_7_extracted)

    with open(f"html/{ticker}_{year}_item_7.txt", "w") as f:
        f.write(item_7_content_html)

    print(f"{ticker} {year} ITEM 7 saved")

    return filing


In [11]:
extract_section("AMZN", 2020)

Output()

AMZN 2020 ITEM 7 saved


╭─────────────────────────────────────── 10-K 📊 filing for AMAZON COM INC ───────────────────────────────────────╮
│                                                                                                                 │
│   Accession Number       Filing Date   Company          CIK                                                     │
│  ───────────────────────────────────────────────────────────────                                                │
│   0001018724-21-000004   2021-02-03    AMAZON COM INC   1018724                                                 │
│                                                                                                                 │
│                                                                                                                 │
│   Links: 🏠 Homepage 📄 Primary Document 📜 Full Submission Text                                                │
│  ────────────────────────────────────────────────────────────────────────